In [1]:
import os, shutil
import numpy as np

from matplotlib import image, pyplot
from skimage.transform import resize


from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout

from keras import callbacks
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, f1_score, recall_score,classification_report,roc_curve, auc 
from sklearn.utils import class_weight

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from PIL import Image,ImageFile
from tensorflow.keras import models
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [2]:
resultPath = 'C:/Users/HP/Downloads/Results/CNN'

In [3]:
DATA_PATH = "C:/Users/HP/Downloads/Image_Hunter_dataset/"
spamData=[]
label=[]
hashList=[]
count=-1
for folder in os.listdir(DATA_PATH):
    print(">>>Reading ",folder)
    count+=1
    
    for file in os.listdir(DATA_PATH+folder):
        if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
            img = image.imread(DATA_PATH+folder+'/'+file)
            hsh = hash(tuple(np.array(img).flatten()))
            if(hsh not in hashList):
                spamData.append(resize(img, (156, 156, 3)))
                hashList.append(hsh)
                label.append(count)
spamData=np.array(spamData)
label=np.array(label)

>>>Reading  NaturalImages
>>>Reading  SpamImages


In [4]:
print("Number of SPAM",len(label[label==0]))
print("Number of HAM",len(label[label==1]))

Number of SPAM 810
Number of HAM 895


In [5]:
print("Spam data shape : ",spamData.shape," Label shape : ",label.shape)

Spam data shape :  (1705, 156, 156, 3)  Label shape :  (1705,)


In [6]:
test_valid_Percentage = 0.3
x_train,x_test,y_train,y_test = train_test_split(spamData,label,test_size = test_valid_Percentage,random_state=42, stratify=label,shuffle=True)

In [7]:
print("x_train shape : ",x_train.shape," y_train shape : ",y_train.shape)
print("x_test shape : ",x_test.shape," y_test shape : ",y_test.shape)

x_train shape :  (1193, 156, 156, 3)  y_train shape :  (1193,)
x_test shape :  (512, 156, 156, 3)  y_test shape :  (512,)


In [8]:
print("Number of train SPAM",len(y_train[y_train==0]))
print("Number of train HAM",len(y_train[y_train==1]))

print("Number of test SPAM",len(y_test[y_test==0]))
print("Number of test HAM",len(y_test[y_test==1]))

Number of train SPAM 567
Number of train HAM 626
Number of test SPAM 243
Number of test HAM 269


In [9]:
y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

print("y_train_oh shape : ",y_train_oh.shape," y_test_oh shape : ",y_test_oh.shape)

y_train_oh shape :  (1193, 2)  y_test_oh shape :  (512, 2)


In [34]:
#Initializing Exception Network as Base Model for Transfer Learning

from tensorflow.keras.applications.resnet50 import ResNet50


base_model_res = ResNet50(input_shape=(156, 156,3), weights='imagenet', include_top=False,classes= 2)

for layer in base_model_res.layers[:143]:
    layer.trainable = False
for layer in base_model_res.layers[144:]:
    layer.trainable = False

for layer in base_model_res.layers:
    print(layer.name , layer.trainable)
len(base_model_res.layers)

input_10 False
conv1_pad False
conv1_conv False
conv1_bn False
conv1_relu False
pool1_pad False
pool1_pool False
conv2_block1_1_conv False
conv2_block1_1_bn False
conv2_block1_1_relu False
conv2_block1_2_conv False
conv2_block1_2_bn False
conv2_block1_2_relu False
conv2_block1_0_conv False
conv2_block1_3_conv False
conv2_block1_0_bn False
conv2_block1_3_bn False
conv2_block1_add False
conv2_block1_out False
conv2_block2_1_conv False
conv2_block2_1_bn False
conv2_block2_1_relu False
conv2_block2_2_conv False
conv2_block2_2_bn False
conv2_block2_2_relu False
conv2_block2_3_conv False
conv2_block2_3_bn False
conv2_block2_add False
conv2_block2_out False
conv2_block3_1_conv False
conv2_block3_1_bn False
conv2_block3_1_relu False
conv2_block3_2_conv False
conv2_block3_2_bn False
conv2_block3_2_relu False
conv2_block3_3_conv False
conv2_block3_3_bn False
conv2_block3_add False
conv2_block3_out False
conv3_block1_1_conv False
conv3_block1_1_bn False
conv3_block1_1_relu False
conv3_block1_2_co

175

In [35]:
NO_OF_EPOCHS=10
BATCH_SIZE=128

model=Sequential()
model.add(base_model_res)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 51200)             0         
                                                                 
 dense_4 (Dense)             (None, 128)               6553728   
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 30,141,569
Trainable params: 7,078,657
Non-trainable params: 23,062,912
_________________________________________________________________


In [36]:
checkpointer = callbacks.ModelCheckpoint(filepath=resultPath+"/checkpoint-{epoch:04d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger(resultPath+'/result_logger.csv',separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.2, patience=2, min_lr=0.001)
model.fit(x_train,y_train,epochs= 10,verbose=1,batch_size= 64,validation_data=(x_test,y_test),callbacks=[checkpointer,csv_logger,reduce_lr])

Epoch 1/10
19/19 [==============================] - 74s 4s/step - loss: 1.0371 - accuracy: 0.5281 - val_loss: 0.6961 - val_accuracy: 0.5293 - lr: 0.0010
Epoch 2/10
19/19 [==============================] - 72s 4s/step - loss: 0.6025 - accuracy: 0.7008 - val_loss: 0.3940 - val_accuracy: 0.9238 - lr: 0.0010
Epoch 3/10
19/19 [==============================] - 96s 5s/step - loss: 0.3929 - accuracy: 0.8625 - val_loss: 0.2240 - val_accuracy: 0.9082 - lr: 0.0010
Epoch 4/10
19/19 [==============================] - 72s 4s/step - loss: 0.4041 - accuracy: 0.8097 - val_loss: 0.2413 - val_accuracy: 0.9180 - lr: 0.0010
Epoch 5/10
19/19 [==============================] - 71s 4s/step - loss: 0.2534 - accuracy: 0.9128 - val_loss: 0.2982 - val_accuracy: 0.8789 - lr: 0.0010
Epoch 6/10
19/19 [==============================] - 71s 4s/step - loss: 0.2073 - accuracy: 0.9313 - val_loss: 0.3070 - val_accuracy: 0.8711 - lr: 0.0010
Epoch 7/10
19/19 [==============================] - 75s 4s/step - loss: 0.2008 - a

In [37]:
prediction_prob1 = model.predict(x_test,verbose=1)
y_pred=np.round(prediction_prob1)

16/16 [==============================] - 18s 1s/step


In [38]:
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred,average='binary'))
print(recall_score(y_test,y_pred,average='binary'))
print(f1_score(y_test,y_pred,average='binary'))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.87109375
0.8029850746268656
1.0
0.890728476821192
[[177  66]
 [  0 269]]
              precision    recall  f1-score   support

           0       1.00      0.73      0.84       243
           1       0.80      1.00      0.89       269

    accuracy                           0.87       512
   macro avg       0.90      0.86      0.87       512
weighted avg       0.90      0.87      0.87       512

